# Sentiment Classification - PhoBERT + Captum

In [38]:
import pandas as pd
from collections import Counter
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from underthesea import word_tokenize, sent_tokenize, text_normalize
from evaluate import load
import torch
import gc


class Colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## 1. Data

In [15]:
df = pd.read_feather('data/facebook_comments.ftr')

# labels
labels = df['sentiment'].unique().tolist()
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
print(f'label: {id2label}')

df['label'] = df['sentiment'].map({'positive': 0, 'negative': 1})
df.drop(columns=['sentiment'], inplace=True)

label: {0: 'positive', 1: 'negative'}


### 1.1 Tokenizer

In [18]:
def apply_word_tokenize(sen):
    sen = " ".join(sen.split())
    sens = sent_tokenize(sen)
    tokenized_sen = []
    for sen in sens:
        tokenized_sen += word_tokenize(text_normalize(sen))
    return ' '.join(['_'.join(words.split(' ')) for words in tokenized_sen])


df['token'] = df['content'].map(lambda x: apply_word_tokenize(x.lower()))
df.drop(columns=['content'], inplace=True)
df.head()

,label,token
0,0,mình cần mua xúc_xích cho chó nên mình đặt và ...
1,1,"mệt_mỏi quá mọi người ơi . j & t , ghn dừng nh..."
2,0,mấy ac nào mà giờ con ham gửi hàng thì chuẩn_b...
3,0,tình_hình kho pi exress - bưu_cục chi_nhánh ch...
4,1,🛑 tất_cả đơn_vị vận_chuyển shopee đã được bật ...


In [19]:
splits = Dataset.from_pandas(df).train_test_split(test_size=0.3)
dataset_test_valid = splits['test'].train_test_split(test_size=0.5)
train_data, val_data, test_data = splits['train'], dataset_test_valid['train'], dataset_test_valid['test']

raw_dataset = DatasetDict({
    'train': splits['train'].shuffle(seed=42),
    'test': dataset_test_valid['train'],
    'valid': dataset_test_valid['test'],
})
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'token'],
        num_rows: 7845
    })
    test: Dataset({
        features: ['label', 'token'],
        num_rows: 1681
    })
    valid: Dataset({
        features: ['label', 'token'],
        num_rows: 1682
    })
})

In [20]:
for i in raw_dataset:
    print(i, Counter(raw_dataset[i]['label']))

train Counter({0: 4722, 1: 3123})
test Counter({0: 992, 1: 689})
valid Counter({0: 1018, 1: 664})


### 1.2 Tokenizer BERT

In [21]:
pretrain_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(pretrain_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
num = 25
token = tokenizer(raw_dataset['train'][num]['token'], padding='max_length', max_length=15, truncation=True)

print(f"{Colors.OKGREEN}Text:{Colors.ENDC} {raw_dataset['train'][num]['token']}")
print(f"{Colors.OKGREEN}Len of token:{Colors.ENDC} {len(token['input_ids'])}")
print(f"{Colors.OKGREEN}Input_ids:{Colors.ENDC} {token['input_ids']}")

Text: có_thể làm_ơn cho shop tự tắt shopee express thân_yêu được không ? đơn đổ hết về spx thân_yêu của các bạn mà shiper không được tới lấy . làm ơn đi shopee .
Len of token: 15
Input_ids: [0, 62, 43025, 13, 9405, 385, 2135, 12231, 36653, 7928, 33157, 14027, 11, 17, 2]


In [34]:
def preprocess_function(examples):
    return tokenizer(examples['token'], padding="max_length", truncation=True, max_length=128)

tokenized_train = raw_dataset['train'].map(preprocess_function, batched=True)
tokenized_valid = raw_dataset['valid'].map(preprocess_function, batched=True)
tokenized_test = raw_dataset['test'].map(preprocess_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [37]:
example = tokenized_train[26]
print(example.keys())
print(tokenizer.decode(example['input_ids']))
print(example['label'])

dict_keys(['label', 'token', 'input_ids', 'token_type_ids', 'attention_mask'])
<s> có_thể làm_ơn cho shop tự tắt shopee express thân_yêu được không? đơn đổ hết về spx thân_yêu của các bạn mà shiper không được tới lấy. làm ơn đi shopee. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
1


## 2. Models

In [39]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrain_name,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    acc = load('accuracy').compute(predictions=predictions, references=labels)
    f1_macro = load('f1').compute(predictions=predictions, references=labels, average='macro')
    f1_weight = load('f1').compute(predictions=predictions, references=labels, average='weighted')
    return {'accuracy': acc, 'F1 macro': f1_macro, 'F1 weighted': f1_weight}

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [9]:
torch.cuda.empty_cache()
gc.collect()

training_args = TrainingArguments(
    output_dir='phobert/sentiment',
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    weight_decay=0.001,
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    num_train_epochs=10,
    fp16=True,
    logging_strategy='epoch',
    save_strategy='epoch',
    evaluation_strategy='epoch',
    # evaluation_strategy='steps',
    # save_steps=200,
    # eval_steps=200,
    # logging_steps=20,
    save_total_limit=2,
    push_to_hub=False,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Kevin\miniconda3\envs\nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7845
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2455
  Number of trainable parameters = 134999810


Step,Training Loss
500,0.422900
1000,0.303500
1500,0.223800
2000,0.180700


Saving model checkpoint to ./sentiment_phobert\checkpoint-500
Configuration saved in ./sentiment_phobert\checkpoint-500\config.json
Model weights saved in ./sentiment_phobert\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./sentiment_phobert\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./sentiment_phobert\checkpoint-500\special_tokens_map.json
added tokens file saved in ./sentiment_phobert\checkpoint-500\added_tokens.json
Saving model checkpoint to ./sentiment_phobert\checkpoint-1000
Configuration saved in ./sentiment_phobert\checkpoint-1000\config.json
Model weights saved in ./sentiment_phobert\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./sentiment_phobert\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ./sentiment_phobert\checkpoint-1000\special_tokens_map.json
added tokens file saved in ./sentiment_phobert\checkpoint-1000\added_tokens.json
Saving model checkpoint to ./sentiment_phobert\checkpoint-1500
Con

TrainOutput(global_step=2455, training_loss=0.25804647845794615, metrics={'train_runtime': 435.5828, 'train_samples_per_second': 90.052, 'train_steps_per_second': 5.636, 'total_flos': 2418874487460000.0, 'train_loss': 0.25804647845794615, 'epoch': 5.0})

In [10]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()